# Algo Trading Example

This example simulates actual trading using the strategy below:
1. We define a SMA period.
2. If the adjusted close price is above the SMA(SMA period) we enter a long position (we place a buy market order).
3. If a long position is in place, and the adjusted close price drops below the SMA(15) we exit the long position (we place a sell market order).

## 1. Install and import appropriate libraries

In [2]:
! pip install pyalgotrade
! pip install --upgrade pandas
! pip install --upgrade pandas-datareader
! pip install yfinance

     --------------------------------------- 11.3/11.3 MB 40.9 MB/s eta 0:00:00
     ------------------------------------- 341.8/341.8 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Chengkf\\anaconda3\\Lib\\site-packages\\~-ndas\\_libs\\algos.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [ ]:
from __future__ import print_function

from pyalgotrade import strategy
from pyalgotrade.barfeed import yahoofeed
from pyalgotrade.technical import ma

import datetime as dt
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
import matplotlib.pyplot as plt

## 2. Initialize start and end dates and set up data reader

In [ ]:
start = dt.datetime(2020, 1, 1)
end = dt.datetime(2022, 4, 14)

In [1]:
yf.pdr_override()

NameError: name 'yf' is not defined

## 3. Fetch data and load into CSV file

In [ ]:
query_str = "AAPL"
df = web.get_data_yahoo(query_str, start, end)

df

In [ ]:
filename = "{}.csv".format(query_str)
with open(filename, 'w') as csv_file:
  df.to_csv(filename)

Open the file viewer and click on "Refresh" to see the generated CSV.

## 4. Create Strategy Template

In [ ]:
class StrategyTemplate(strategy.BacktestingStrategy):
  def __init__(self, feed, instrument, smaPeriod):        # Initialization for the strategy
    super(StrategyTemplate, self).__init__(feed, 1000)
    self.__position = None
    self.__instrument = instrument
    self.setUseAdjustedValues(True)
    self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

  def onEnterOk(self, position):                          # Actions taken if the program enters a position
    execInfo = position.getEntryOrder().getExecutionInfo()
    self.info("BUY at ${}".format(execInfo.getPrice()))
  
  def onEnterCanceled(self, position):                    # Actions taken if the program cancels entering the position
      self.__position = None
  
  def onExitOk(self, position):                           # Actions taken if the program exits a position
    execInfo = position.getExitOrder().getExecutionInfo()
    self.info("SELL at ${}".format(execInfo.getPrice()))
    self.__position  = None

  def onExitCancelled(self, position):                    # Actions taken if the program cancels exiting a position
    self.__position.exitMarket()
  
  def onBars(self, bars):                                 # Actions taken after reading each row
    if self.__sma[-1] is None:
      return

    bar = bars[self.__instrument]

    if self.__position is None:
      if bar.getPrice() > self.__sma[-1]:
        self.__position = self.enterLong(self.__instrument, 10, True)
    
    elif bar.getPrice() < self.__sma[-1] and not self.__position.exitActive():
      self.__position.exitMarket()

## 5. Define Function for Running Strategy

In [ ]:
def run_strategy(smaPeriod):
  feed = yahoofeed.Feed()
  feed.addBarsFromCSV(query_str, filename)

  myStrategy = StrategyTemplate(feed, query_str, smaPeriod)
  myStrategy.run()
  print("Final portfolio value: ${}".format(myStrategy.getBroker().getEquity()))
  return myStrategy.getResult()

## 6. Run the Strategy

In [ ]:
strategy_results = []
labels = []
for i in range(10,30):
  strategy_results.append(run_strategy(i))
  labels.append(i)

## 7. Display the Results

In [ ]:
plt.figure(figsize=(20,7))
plt.bar(labels, strategy_results)
plt.xticks(labels)